In [2]:
import math
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
#from selenium.webdriver import browser
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from difflib import SequenceMatcher
import datetime
from selenium.webdriver.firefox.options import Options
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
#get_ipython().magic('matplotlib inline')


# In[3]:

#The name of the csv file to be opened
#readDataFileLoc = "scopus_2000_Citation.csv"


# In[4]:

#Reading the csv file and doing the required preprocessing
#It takes only the required columns - Authors, Title, Year, Cited by
#Removing the rows i.e. papers whose citations are 0
citationDataset = pd.read_csv("data.csv",)
df = pd.read_csv("data.csv",)
#citationDataset=citationDataset1.iloc[:10,:]
citationDataset.head()
#newCitationDataset = citationDataset[['Authors', 'Title', 'Year', 'Cited by']]
#print(newCitationDataset.head())
#finalCitationDataset = newCitationDataset[pd.notnull(newCitationDataset['Cited by'])]
#listCitationData = finalCitationDataset.values
#print(len(listCitationData))
#print(finalCitationDataset)


# In[9]:

def getScholarSearchResult(browser, queryToSearch):
    """
    To type the query in the google scholar search bar and search it

    Args:
        browser: An open browser using selenium(preferably firefox)
        queryToSearch: A string which has to be searched in the google scholar

    Returns:
        browser(no need actually)
    """
    scholarSearchBar = browser.find_element_by_name("q")
    scholarSearchBar.send_keys(queryToSearch)
    scholarSearchBar.submit()
    return browser

def searchScholarResults(browser, queryToSearch):
    """
    To type the query in the google scholar search bar and search it

    Args:
        browser: An open browser using selenium(preferably firefox)
        queryToSearch: A string which has to be searched in the google scholar

    Returns:
        Nothing
    """
    scholarSearchBar = browser.find_element_by_name("q")
    scholarSearchBar.send_keys(queryToSearch)
    scholarSearchBar.submit()
    return

def customSimilarityChecker(listpapers, paperTitle):
    """
    A basic similarity checker which compares all the Names of paper displayed by the Scholar with the desired paper and clicks the link of the paper

    Args:
        listpapers: HTML listofpapers which contain the Title of the papers
        paperTitle: Title of the paper which we are searching

    Returns:
        Nothing
    """
    index = 0
    maxDiff = SequenceMatcher(None, listpapers[0].text, paperTitle).ratio()
    #print(maxDiff)
    for x in range(0, len(listpapers)):
        temp = SequenceMatcher(None, listpapers[x].text, paperTitle).ratio()
        if temp > maxDiff:
            index = x
            maxDiff = temp
    listpapers[index].click()
    return

def getYearCitationInformation(browser):
    """
    Calling this function will return an 1-d array containing pairs of year and citations in that year of the paper
    Takes the open page of the paper in Google Scholar as input

    Args:
        browser: An open browser using selenium(preferably firefox)

    Returns:
        A 1-d array containing pairs of year and citations in that year of the paper
    """
    answer = []
    try:
        idBars = browser.find_element_by_id('gsc_vcd_graph_bars')
    except Exception:
        return answer
    barsList = idBars.find_elements_by_tag_name('a')
    lengthBars = len(barsList)
    listOfBarsYear = idBars.find_elements_by_class_name('gsc_vcd_g_t')
    for i in range(0, lengthBars):
        answer.append([(barsList[lengthBars - i - 1].find_elements_by_tag_name('span'))[0].get_attribute('textContent'),
                       listOfBarsYear[lengthBars - i - 1].text])
    
    return answer

#This function will click the more button repeteadly to display all the research papers published by a person on the page
def correctClickMoreButton(browser):
    """

    Args:
        browser: 

    :return:
    """
    return

def searchChooseCitationInfo(browser, queryToSearch, paperTitle):
    """
    Calling this function will return an 1-d array containing pairs of year and citations in that year of the paper,
     taking only an open browser as input
    :param browser: An open browser using selenium(preferably firefox)
    :param queryToSearch: The text query which we are going to search in the google scholar
    :param paperTitle: Title of the paper which we are searching
    :return: A 1-d array containing pairs of year and citations in that year of the paper
    """
    browser.get("https://scholar.google.com/")
    scholarSearchBar = browser.find_element_by_name("q")
    scholarSearchBar.send_keys(queryToSearch)
    scholarSearchBar.submit()
    #tempList = browser.find_elements_by_class_name("gs_a")
    #tempList2 = tempList[0].find_elements_by_tag_name('a')
    #tempList2[0].click()
    time.sleep(5)
    ((browser.find_elements_by_class_name("gs_a"))[0].find_elements_by_tag_name('a'))[0].click()
    time.sleep(5)
    correctClickMoreButton(browser)
    listpapers = browser.find_elements_by_class_name('gsc_a_at')
    customSimilarityChecker(listpapers, paperTitle)
    time.sleep(5)
    return getYearCitationInformation(browser)




def writeListCitationToCsv(fileName, answer, listCitationData, score):
    """
    Writes the citation data to the csv file

    Args:
        fileName: name of the file to which data has to be written
        answer: citation data for each paper with respect to time
        listCitationData: consist of related data of each paper
        score: score for each paper

    Returns:
        Nothing
    """
    max = 0
    now = datetime.datetime.now()
    start = int(now.year)
    i = 0
    for i in range(0, len(answer)):
        if len(answer[i]) > 0:
            t = start - int(answer[i][0][1]) + len(answer[i])
        else:
            t = len(answer[i])
        if t > max:
            max = t
    f = open(fileName, 'w+')
    csvout = csv.writer(f)
    row = []
    row.extend(['Authors', 'Title', 'Year',
                'Cited by', 'Cited by updated', 'Last Citation Year', 'Score'])
    rowInitData = len(row)
    for i in range(0, max):
        row.extend([start - i])
    end = start - max + 1
    csvout.writerow(row)
    for i in range(0, len(answer)):
        row = []
        sum = 0
        for m in range(0, len(answer[i])):
            sum = sum + int(answer[i][m][0])
        if len(answer[i]) > 0:
            row.extend([listCitationData[i][0], listCitationData[i][1], listCitationData[i][2]
                           , listCitationData[i][3], sum, answer[i][len(answer[i]) - 1][1], score[i]])
        else:
            row.extend([listCitationData[i][0], listCitationData[i][1], listCitationData[i][2]
                           , listCitationData[i][3], 0, -1, score[i]])
        j = 0
        k = start
        while ((j < len(answer[i])) and (k >= end)):
            if (int(answer[i][j][1]) == k):
                row.extend([answer[i][j][0]])
                j = j + 1
            else:
                row.extend([0])
            k = k - 1
        incompIter = max - len(row) + rowInitData
        for l in range(0, incompIter):
            row.extend([0])
        csvout.writerow(row)
    return


# In[6]:

#Opening the browser
#self.driver = webdriver.Firefox()
#opts = webdriver.chrome.options.Options()
#opts.set_headless()
#assert opts.headless
browser = webdriver.Chrome("chromedriver.exe")
#browser = webdriver.Chrome(options=opts)
browser.get("https://scholar.google.com/")

# In[7]:

#Collect the data
answer = []
for i in range(0, len(df)):
    paperTitle = df["Source title"][i]
    
    #paperAuthor = listCitationData[i][0]
    try:
        answer.append(searchChooseCitationInfo(browser, paperTitle , paperTitle))
    except Exception:
        answer.append([])
    #print(i)
browser.quit()

In [4]:
(barsList[lengthBars - i - 1].find_elements_by_tag_name('span'))[0].get_attribute('textContent')
answer[1]

[['2', '2021'],
 ['11', '2020'],
 ['11', '2019'],
 ['8', '2018'],
 ['17', '2017'],
 ['19', '2016'],
 ['21', '2015'],
 ['28', '2014'],
 ['34', '2013'],
 ['25', '2012'],
 ['18', '2011'],
 ['14', '2010'],
 ['15', '2009'],
 ['10', '2008'],
 ['4', '2007'],
 ['5', '2006'],
 ['7', '2005'],
 ['2', '2004']]

In [1]:
import mysql.connector
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)

In [3]:
cursor = db.cursor()

query = ("SELECT * FROM users")
cursor.execute(query)

In [42]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)
cursor = db.cursor()
query = ("SELECT * FROM users")
cursor.execute(query)
for (id, UserID, Email, Password) in cursor:
    print(id)
    print(UserID)
    print(Email)
    print(Password)
cursor.close()

1
aritra
aritrashome.shome8@gmail.com
1
3
xyz
xyz@gmail.com
1


True

In [70]:
SCA = 60

In [71]:
import pandas as pd
#df = pd.read_csv('main/data.csv')
userid = 'aritra'
"INSERT INTO `papers` (`id`, `title`, `UserID`, `KDM`, `CAOT`, `SQM`, `SCA`, `topic`, `time`) VALUES (NULL, '"+title+"','"+UserID+"', '"+str(KDM)+"','"+str(CAOT)+"','"+str(SQM)+"','"+str(SCA)+"','"+str(topic)+"', current_timestamp());"

"INSERT INTO `papers` (`id`, `title`, `UserID`, `KDM`, `CAOT`, `SQM`, `SCA`, `topic`, `time`) VALUES (NULL, 'Computer recreations','aritra', '0','-1000.0','-1000','60','3', current_timestamp());"

In [72]:
################### READ ###################
i=0
title =df['Source title'][i] 
UserID=userid
KDM=df['KDM'][i] 
CAOT=df['CAOT'][i]
SQM=df['SQM'][i]
SCA=df['SCA'][i]
topic=df['Topic'][i]

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)
cursor = db.cursor()
sql = ("INSERT INTO `papers` (`id`, `title`, `UserID`, `KDM`, `CAOT`, `SQM`, `SCA`, `topic`, `time`) VALUES (NULL, '"+title+"','"+UserID+"', '"+str(KDM)+"','"+str(CAOT)+"','"+str(SQM)+"','"+str(SCA)+"','"+str(topic)+"', current_timestamp());")
#val = (title, UserID, KDM, CAOT, SQM, SCA, topic)
cursor.execute(sql)
db.commit()
cursor.close()
#print(title)

True

In [55]:
################## SEARCH ##########################
userid = 'aritra'
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)
cursor = db.cursor()
sql = ("SELECT * FROM papers WHERE UserID='"+str(userid)+"'")
cursor.execute(sql)
res = cursor.fetchall()
#for i in range(len(res)):
cursor.close()
res

[(1,
  'Computers',
  'aritra',
  11.0,
  5.0,
  51.0,
  23.0,
  1,
  datetime.datetime(2021, 5, 16, 20, 18, 5))]

In [58]:
res[0][1]

'Computers'

In [46]:
###################### Login ###############################
userid = 'x'
password = '1'
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)
cursor = db.cursor()
query = ("SELECT * FROM users WHERE UserID='"+str(userid)+"'")
cursor.execute(query)

res = cursor.fetchall()
if(len(res)==0): 
    print('wrong user')
    

elif(res[0][3]!=password):
    print('wrong pass')
else: print(id)

cursor.close()

14


In [74]:
######################## Register ##############################
userid = 'x1'
email = 'x1@abcd.com'
password = '1'

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)
cursor = db.cursor()
sql = ("SELECT * FROM users WHERE UserID='"+str(userid)+"'")
cursor.execute(sql)
res = cursor.fetchall()
if(len(res)!=0): 
    print('Already user')
    
else:
    insert = ("INSERT INTO `users` (`id`, `UserID`, `email`, `password`) VALUES (NULL, '"+userid+"', '"+email+"', '"+password+"');")
    cursor.execute(insert)
    db.commit()
cursor.close()

True

In [28]:
#Register
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='db'
)
cursor = db.cursor()
insert = ("INSERT INTO `users` (`id`, `UserID`, `email`, `password`) VALUES (NULL, %s, %s, %s);")
cursor.execute(insert,(UserID,email,password))

cursor.close()

True

In [19]:
from __future__ import print_function
import numpy as np
from itertools import combinations, permutations
import pandas as pd

df = pd.read_csv('main/data.csv')
ranks = [df['CAOT'],df['SQM']]

In [20]:
def kendalltau_dist(rank_a, rank_b):
    tau = 0
    n_candidates = len(rank_a)
    for i, j in combinations(range(n_candidates), 2):
        tau += (np.sign(rank_a[i] - rank_a[j]) ==
                -np.sign(rank_b[i] - rank_b[j]))
    return tau
def rankaggr_brute(ranks):
    min_dist = np.inf
    best_rank = None
    n_voters, n_candidates = ranks.shape
    for candidate_rank in permutations(range(n_candidates)):
        dist = np.sum(kendalltau_dist(candidate_rank, rank) for rank in ranks)
        if dist < min_dist:
            min_dist = dist
            best_rank = candidate_rank
    return best_rank

In [21]:
np.array(ranks).shape

(2, 11)

In [22]:
aggr = rankaggr_brute(np.array(ranks))

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  del sys.path[0]


KeyboardInterrupt: 

In [2]:
df.head()

,Unnamed: 0,Link,Source title,Index Keywords,Abstract,KDM,CAOT,SQM,SCA,Topic
0,0,http://paulbourke.net/fractals/peterdejong/pet...,Computer recreations,zones; xsin; x; wrote; write,COMPUTER RECREATIONS Author s A. K. Dewdney ...,0,-1000.000000,-1000,60,3
1,1,http://www.ivanpoupyrev.com/e-library/2004/CHI...,Gummi: a bendable computer,zooming; zoom; would; world; work,Gummi A Bendable Computer Carsten Schwesig ...,0,5.122833,56,57,3
2,2,http://www.cs.cmu.edu/afs/cs/academic/class/15...,[BOOK][B] The design and analysis of computer ...,zs; z; would; working; words,THE DESIGN AND ANALYSIS OF COMPUTER ALGO...,0,774.245250,-32740,57,5
3,3,http://gordonbell.azurewebsites.net/tcmwebpage...,The illiac iv computer,zorz; zoro; zorn; zor; zerxta,Chapter 27 The ILLIAC IV computer 1 George H...,0,-1000.000000,-1000,55,3
4,4,http://wiki.cgt3d.cn/mediawiki/images/f/f7/Com...,[BOOK][B] Computer Systems,zp; zone; zombie; zhang; zf,third global edition Computer Systems A Prog...,0,-1000.000000,-1000,54,3


In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd 

from app import db,User,Papers

Using TensorFlow backend.
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [2]:
user = 'aritra'

papers = Papers.query.filter_by(UserID=user).all()
len(papers)

18

In [3]:
inputs = 10
Xin = []
Yin = []
for i in range(len(papers)-inputs):
    Xi = []
    for j in range(inputs):
        X = [papers[i+j].KDM, papers[i+j].SCA, papers[i+j].SQM, papers[i+j].CAOT,0,0,0,0,0]
        n = papers[i+j].topic
        try:
            X[3+n] = 1
        except:
            X[3+int.from_bytes(n,"little" )] = 1
        Xi.append(X)
    Xin.append(Xi)
for i in range(inputs,len(papers)):
    Yi = [0,0,0,0,0]
    try:
        Yi[papers[i].topic - 1] = 1
    except:
        Yi[int.from_bytes(papers[i].topic,"little" ) - 1] = 1
    Yin.append(Yi)

Yin = np.array(Yin)
Xin = np.array(Xin)
print(Xin.shape)
print(Yin.shape)

(8, 10, 9)
(8, 5)


In [4]:
model = Sequential()
model.add(LSTM(50,return_sequences=True, input_shape=(Xin.shape[1], Xin.shape[2])))
model.add(LSTM(10,return_sequences=False))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 50)            12000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
Total params: 14,495
Trainable params: 14,495
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.fit(Xin, Yin, epochs = 5, shuffle=False,)

Epoch 1/5
8/8 [==============================] - 2s 207ms/step - loss: 1.6017 - accuracy: 0.1250
Epoch 2/5
8/8 [==============================] - 0s 749us/step - loss: 1.5583 - accuracy: 0.3750
Epoch 3/5
8/8 [==============================] - 0s 925us/step - loss: 1.5231 - accuracy: 0.5000
Epoch 4/5
8/8 [==============================] - 0s 997us/step - loss: 1.4947 - accuracy: 0.5000
Epoch 5/5
8/8 [==============================] - 0s 997us/step - loss: 1.4691 - accuracy: 0.6250


In [11]:
final = np.argsort((-model.predict(Xin[-1].reshape(1,inputs,Xin.shape[2]))))[0][:2] + 1

In [14]:
final

array([3, 4], dtype=int64)

5